In [16]:
import sagemaker
import boto3
%pip install --upgrade boto3
from sagemaker import image_uris
# from sagemaker.amazon.amazon_estimator import ImageURIProvider
from sagemaker.session import s3_input,Session

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
bucket_name='bank-demo-123'
my_region=boto3.session.Session().region_name
print(my_region)

us-east-1


In [3]:
s3=boto3.resource('s3')
try:
    if my_region=='us-east-1':
        s3.create_bucket(Bucket=bucket_name)
        print("Bucket created Sucessfully")
except Exception as e:
    print("s3 error",e)

Bucket created Sucessfully


In [4]:
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://bank-demo-123/xgboost-as-a-built-in-algo/output


In [6]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [7]:
import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [12]:
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
# upload the dataframe as csv
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [13]:
pd.concat([test_data['y_yes'],test_data.drop(['y_no','y_yes'],axis=1)],axis=1).to_csv('test.csv',index=False,header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix,'test/test.csv')).upload_file('test.csv')
s3_input_test=sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

In [14]:
type(s3_input_test)

sagemaker.inputs.TrainingInput

In [15]:

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "verbosity":"1",
        "objective":"reg:squarederror",
        "num_round":"50"}
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "1.5-1")



In [18]:
pwd

'/home/ec2-user/SageMaker'

In [19]:
# construct a SageMaker estimator that calls the xgboost-container
# estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container, 
#                                           hyperparameters=hyperparameters,
#                                           role=sagemaker.get_execution_role(),
#                                           train_instance_count=1, 
#                                           train_instance_type='ml.m5.2xlarge', 
#                                           train_volume_size=5, # 5 GB 
#                                           output_path=output_path,
#                                           train_use_spot_instances=True,
#                                           train_max_run=300,
#                                           train_max_wait=600)
# set an output path to save your trained model

estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path)

In [20]:


estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-02-15-19-38-18-839


2023-02-15 19:38:19 Starting - Starting the training job...
2023-02-15 19:38:34 Starting - Preparing the instances for training...
2023-02-15 19:39:19 Downloading - Downloading input data...
2023-02-15 19:39:44 Training - Downloading the training image...
2023-02-15 19:39:59 Training - Training image download completed. Training in progress..[2023-02-15 19:40:18.113 ip-10-2-72-78.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-02-15:19:40:18:INFO] Imported framework sagemaker_xgboost_container.training
[2023-02-15:19:40:18:INFO] Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
[2023-02-15:19:40:18:INFO] No GPUs detected (normal if no gpus installed)
[2023-02-15:19:40:18:INFO] Running XGBoost Sagemaker in algorithm mode
[2023-02-15:19:40:18:INFO] Determined delimiter of CSV input is ','
[2023-02-15:19:40:18:INFO] Determined delimiter of CSV input is ','
[2023-02-15:19:40:18:INFO] files path: /opt/ml/input/data

In [21]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2023-02-15-19-43-50-865
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2023-02-15-19-43-50-865
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2023-02-15-19-43-50-865


--------!

In [29]:
from sagemaker.serializers import CSVSerializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
# xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer =  CSVSerializer()# set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!


In [30]:
predictions

'0.05101925879716873\n0.05101925879716873\n0.05101925879716873\n0.33061444759368896\n0.08849576860666275\n0.08849576860666275\n0.05101925879716873\n0.05101925879716873\n0.040609098970890045\n0.040609098970890045\n0.08849576860666275\n0.07054923474788666\n0.05101925879716873\n0.08849576860666275\n0.08849576860666275\n0.05101925879716873\n0.08849576860666275\n0.08849576860666275\n0.08849576860666275\n0.08849576860666275\n0.05101925879716873\n0.0637093260884285\n0.05101925879716873\n0.08849576860666275\n0.08849576860666275\n0.08849576860666275\n0.08849576860666275\n0.040609098970890045\n0.05101925879716873\n0.05101925879716873\n0.1291365772485733\n0.05101925879716873\n0.08849576860666275\n0.05101925879716873\n0.05101925879716873\n0.07054923474788666\n0.14635741710662842\n0.05101925879716873\n0.2397281378507614\n0.08849576860666275\n0.07054923474788666\n0.07054923474788666\n0.05101925879716873\n0.07054923474788666\n0.06824012100696564\n0.08849576860666275\n0.08849576860666275\n0.0884957686

In [31]:
predictions_array = np.fromstring(predictions[1:], sep='\n') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [32]:

cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.6%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10826)    30% (110)
Purchase        10% (1169)     70% (252) 



In [33]:

sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2023-02-15-19-43-50-865


[{'ResponseMetadata': {'RequestId': 'VY0Z1XM3YS38SA95',
   'HostId': '4qcV6clKNBOo98yyAWmn+2L30sv6k+gNpsNZz7E5fasQgE3DPP4BCgHlZ08Y2iYKwrmrHUJPCEc=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': '4qcV6clKNBOo98yyAWmn+2L30sv6k+gNpsNZz7E5fasQgE3DPP4BCgHlZ08Y2iYKwrmrHUJPCEc=',
    'x-amz-request-id': 'VY0Z1XM3YS38SA95',
    'date': 'Wed, 15 Feb 2023 20:11:10 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/train/train.csv'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-02-15-19-38-18-839/profiler-output/system/incremental/2023021519/1676490000.algo-1.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-02-15-19-38-18-839/debug-output/training_job_end.ts'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-02-15-19-38-18-839/output/model.tar.gz'},
   {'K